# Nutrisense

In [1]:
! pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 578.7 kB/s eta 0:00:19
   - -------------------------------------- 0.5/11.1 MB 578.7 kB/s eta 0:00:19
   - -------------------------------------- 0.5/11.1 MB 578.7 kB/s eta 0:00:19
   - -------------------------------------- 0.5/11.1 MB 578.7 kB/s eta 0:00:19
   -- ------------------------------------- 0.8/11.1 MB 419.4 kB/s eta 0:00:25
   --- ------------------------------------ 1.0/11.1 MB 503.5 kB/s eta 0:00:20
   --- ------------------------------------ 1.0/11.1 MB 503.5 kB/s eta 0:00:20
   ---- -----------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [35]:
import pandas as pd

# Load dataset
file_path = 'reduced_dataset.csv'
recipe_df = pd.read_csv(file_path)

recipe_df.head()

,Unnamed: 0,recipe_id,recipe_name,aver_rate,image_url,review_nums,calories,fat,carbohydrates,protein,cholesterol,sodium,fiber,ingredients_list
0,19591,245720,Churro Log Cabin,5.00,http://images.media-allrecipes.com/userphotos/...,1,77,70,91,10,2,15,9,"['canned white frosting', ""pre-made churros (s..."
1,11552,247451,Pulled BBQ Chicken Sliders,4.17,https://images.media-allrecipes.com/userphotos...,5,20,20,110,32,12,30,4,"['skinless boneless chicken breast halves', 'o..."
2,4780,232860,Easy Tasty Ham Steaks with Maple Glaze For 2,4.62,https://images.media-allrecipes.com/userphotos...,19,52,18,71,36,15,65,10,"['ham steak', 'brown sugar', 'maple syrup', 'D..."
3,24585,245028,Poached Pears Belle Helene,5.00,http://images.media-allrecipes.com/userphotos/...,7,53,21,78,8,5,10,35,"['water', 'lemon zested and juiced', 'Bosc pea..."
4,20998,232909,American Flag Cake,4.22,http://images.media-allrecipes.com/userphotos/...,38,55,39,69,21,3,31,2,"['white cake mix', 'egg whites', 'vegetable oi..."


In [36]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [37]:
# Preprocess bahan
vectorizer = TfidfVectorizer()
X_ingredients = vectorizer.fit_transform(recipe_df['ingredients_list'])

In [38]:
# Normalisasi data numerik
scaler = StandardScaler()
X_numerical = scaler.fit_transform(recipe_df[['protein', 'carbohydrates', 'fat', 'fiber']])

In [39]:
# gabungkan fitur
X_combined = np.hstack([X_numerical, X_ingredients.toarray()])

# Train KNN Model dgn n = 3
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X_combined)

NearestNeighbors(metric='euclidean')

In [40]:
import pickle
pickle.dump(knn, open('reduced_recommendation_model.pkl' , 'wb'))
pickle.dump(scaler, open('scaler.pkl', 'wb'))
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))

In [26]:
import pickle
knn = pickle.load(open('reduced_recommendation_model.pkl', 'rb'))
scaler = pickle.load(open('scaler.pkl', 'rb'))
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))

In [42]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

# Function to Recommend Recipes
def recommend_recipes(umur, gender, list_bahan):
    #gizi sesuai PERATURAN MENTERI KESEHATAN REPUBLIK INDONESIA
    #               protein, karbo, fat,  fiber
    data_nutrisi = [[13.3,   83.3,  18.3, 7.8], #kategori 0
                    [16.7,   100,   21.7, 9.3], #kategori 1
                    [18.3,   93.3,  21.7, 9  ]] #kategori 2
    
    #mengategorikan berdasarkan umur dan gender
    #dikhusus untuk anak sd range 7-12
    if umur < 7:
        print("maaf umur tidak termasuk anak sd")
    elif umur < 10:
        kategori = 0
    elif umur < 13:
        if gender == 1: #1 untuk laki laki dan 0 untuk perempuan
            kategori = 1
        else:
            kategori = 2
    else:
        print("maaf umur tidak termasuk anak sd") 
    
    #mengisi input feature berdasarkan kategori
    input_features = []
    for i in range(0,4):
        input_features.append(data_nutrisi[kategori][i])
    input_features.append(list_bahan)
    
    # merekomendasikan resep
    input_features_scaled = scaler.transform([input_features[:4]])
    input_ingredients_transformed = vectorizer.transform([input_features[4]])
    input_combined = np.hstack([input_features_scaled, input_ingredients_transformed.toarray()])
    distances, indices = knn.kneighbors(input_combined)
    recommendations = recipe_df.iloc[indices[0]]

    #on progress:)
    # # menyimpan data nutrisi satu persatu untuk dibandingkan
    # nutrisi_resep = []
    # selector = ['protein', 'carbohydrates', 'fat', 'fiber']
    # for i in range(0,4):
    #     nutrisi_resep.append(recipe_df.iloc[indices[0][1]][selector[i]])

    # # membandingkan nutrisi resep dengan acuan
    # gizi_terpenuhi = []
    # for i in range(0,4):
    #     gizi_terpenuhi.append(nutrisi_resep[i] >= data_nutrisi[kategori][i])

    # #print gizi yang terpenuhi
    # for i in range(0, 4):
    #     print(f'{selector[i]} : {gizi_terpenuhi[i]}')
    #return rekomendasi
    return recommendations[['recipe_name', 'ingredients_list', 'image_url']]

# Example Input
input_features = [10, 1,'chicken']
recommendations = recommend_recipes(10, 1,'chicken')
recommendations

c:\Users\ergik\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,recipe_name,ingredients_list,image_url
1,Pulled BBQ Chicken Sliders,"['skinless boneless chicken breast halves', 'o...",https://images.media-allrecipes.com/userphotos...
0,Churro Log Cabin,"['canned white frosting', ""pre-made churros (s...",http://images.media-allrecipes.com/userphotos/...
3,Poached Pears Belle Helene,"['water', 'lemon zested and juiced', 'Bosc pea...",http://images.media-allrecipes.com/userphotos/...
2,Easy Tasty Ham Steaks with Maple Glaze For 2,"['ham steak', 'brown sugar', 'maple syrup', 'D...",https://images.media-allrecipes.com/userphotos...
4,American Flag Cake,"['white cake mix', 'egg whites', 'vegetable oi...",http://images.media-allrecipes.com/userphotos/...
